In [1]:
import os
import re # Regular expressions

In [2]:
# Variable declaration
image_mode = "FLAIR" # FLAIR - T1w - T1wCE - T2w
patient_mode = "central" # central - first TODO: Explain 
input_dir = "../input/rsna-miccai-png/train" 
target_dir = "../input/rsna-miccai-png/test"

In [3]:
# Get path to the training images
input_img_paths_patients = sorted([
    os.path.join(input_dir, fname + "/" + image_mode) 
    for fname in os.listdir(input_dir)
    if os.path.exists(os.path.join(input_dir, fname + "/" + image_mode))])

input_img_paths = []
for patient in input_img_paths_patients:
    # TODO: Explain
    # TODO: Choosing first makes no sense. Because of the way images are taken. Center or bigger
    images_id = x = [int(re.findall(r'\d+', x)[0]) for x in os.listdir(patient)] # Get all image ids
    if patient_mode == "central":
        index_central_image = min(images_id) + round((max(images_id) - min(images_id))/2) # Get middle image
        path = os.path.join(patient, "Image-" + str(index_central_image) + ".png")
        if os.path.exists(path): input_img_paths.append(path)
    if patient_mode == "first":
        index_central_image = min(images_id) # Get first image
        path = os.path.join(patient, "Image-" + str(index_central_image) + ".png")
        if os.path.exists(path): input_img_paths.append(path)